In [15]:
import numpy as np
import h5py
from glob import glob
from tqdm import tqdm
import os

In [43]:
src = 'datasets/2024_6to6/2024_3dvq_replicate'
tgt = 'datasets/2024_6to6/2024_3dvq_replicate_aggregated_chunk'

h5files = list(sorted(glob(f'{src}_1_*.h5')))

In [44]:
h5files

['datasets/2024_6to6/2024_3dvq_replicate_1_0.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_1.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_2.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_3.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_4.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_5.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_6.h5',
 'datasets/2024_6to6/2024_3dvq_replicate_1_7.h5']

In [45]:
data_all = []
check_all = []
datestr_all = []
for name in tqdm(h5files):
    with h5py.File(name, 'r', rdcc_nbytes=1024**2*2048, rdcc_nslots=1e7) as f:
        data_all.append(f['data'][:])
        check_all.append(f['check'][:])
        datestr_all.append(f['datestr'][:])

100%|██████████| 8/8 [00:03<00:00,  2.05it/s]


In [46]:
data = np.concatenate(data_all, axis=0)
check = np.concatenate(check_all, axis=0)
datestr = np.concatenate(datestr_all, axis=0)

In [47]:
datestr2idx = {d:i for i, d in enumerate(datestr)}

In [48]:
sorted_datestr2idx = dict(sorted(datestr2idx.items()))

In [49]:
non_overlapping_data = []
non_overlapping_check = []
non_overlapping_datestr = []

for k, v in sorted_datestr2idx.items():
    non_overlapping_data.append(data[v])
    non_overlapping_check.append(check[v])
    non_overlapping_datestr.append(k)

In [50]:
non_overlapping_data = np.stack(non_overlapping_data, axis=0)
non_overlapping_check = np.array(non_overlapping_check, dtype=bool)
non_overlapping_datestr = np.array(non_overlapping_datestr, dtype='S')

In [51]:
print(non_overlapping_data.shape)

(9013, 72, 16, 16)


In [52]:
print(f'{tgt}.h5')
with h5py.File(f'{tgt}.h5', 'w') as f:
    # f.create_dataset('data', data=non_overlapping_data, compression="gzip", chunks=(48, 48, 16, 16), maxshape=(None, 48, 16, 16))
    f.create_dataset('data', data=non_overlapping_data, compression="lzf", chunks=(1, 72, 16, 16), maxshape=(None, 72, 16, 16))
    # f.create_dataset('datestr', data=non_overlapping_datestr, compression="gzip", chunks=True, maxshape=(None, ))
    f.create_dataset('datestr', data=non_overlapping_datestr, compression="lzf", chunks=True, maxshape=(None, ))
    # f.create_dataset('check', data=non_overlapping_check, compression="gzip", chunks=True, maxshape=(None, ))
    f.create_dataset('check', data=non_overlapping_check, compression="lzf", chunks=True, maxshape=(None, ))
    f.close()

datasets/2024_6to6/2024_3dvq_replicate_aggregated_chunk.h5


In [53]:
# # write to txt
# with open(f'{tgt}.txt', 'w') as f:
#     for i in range(len(non_overlapping_datestr)):
#         f.write(f"{non_overlapping_datestr[i].decode('utf-8')}\n")

In [54]:
f.close()